# Few-shot Learning

在开始之前，我们先回顾一下我们之前所写的 prompts，我们在指示模型去回答我们的问题时，有没有给它提供参考的例子？答案是没有，我们之前说过，这是一种 Zero-shot Learning。那么什么是 Few-shot Learning？我们现在就用一个 Zero-shot prompt 来提问模型，让模型告诉我们答案，顺便重新体会一下 Zero-shot Learning 的概念。

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(
    model="deepseek-chat",
    base_url="https://api.deepseek.com",
    api_key=os.getenv("DEEPSEEK_API_KEY")
)

prompt = "什么是 Few-shot learning?"

print(llm.invoke(prompt).content)

相信你对 Few-shot Learning 已经有一定的概念了，那么接下来我们会从文本情感分析任务来研究一下它。

我们会给模型一些例子，一句话到底是积极的还是消极的，模型会通过这些例子来学习，然后对一句没见过的话进行判断。为了给模型上点强度，我们决定使用古诗词作为数据。

In [ ]:
def few_shot_sentiment_classification(input_text):
    few_shot_prompt = PromptTemplate(
        input_variables=["input_text"],
        template="""
        文本：力拔山兮气盖世，时不利兮骓不逝。
        情感：消极

        文本：邂逅相遇，与子偕臧。
        情感：积极

        文本：匪女之为美，美人之贻。
        情感：积极

        文本：{input_text}
        情感：
        """
    )

    chain = few_shot_prompt | llm
    result = chain.invoke(input_text).content

    return result

test_text = "袅袅兮秋风，洞庭波兮木叶下。"
result = few_shot_sentiment_classification(test_text)
print(f"输入：{test_text}")
print(f"情感预测：{result}")

这时候你可能会想，如果给出的示例是随机的，并不一定对应正确的答案，模型是否可以正确预测？接下来我们就来看看。

In [ ]:
def few_shot_sentiment_classification(input_text):
    few_shot_prompt = PromptTemplate(
        input_variables=["input_text"],
        template="""
        文本：欲穷千里目，更上一层楼。
        情感：消极

        文本：风飒飒兮木萧萧，思公子兮徒离忧。
        情感：积极

        文本：袅袅兮秋风，洞庭波兮木叶下。
        情感：积极

        文本：{input_text}
        情感：
        """
    )

    chain = few_shot_prompt | llm
    result = chain.invoke(input_text).content

    return result

test_text = "夜中不能寐，起坐弹鸣琴。"
result = few_shot_sentiment_classification(test_text)
print(f"输入：{test_text}")
print(f"情感预测：{result}")

事实上，模型远比你想象的强大，即使这样它还是可以回答正确，而这里最重要的是，是否有一个标签提供了情感这一条信息，至于标签是否随机化，这对于效果可能会存在影响，但现在看来模型的能力还没有到随便一个例子随机化标签都会答错这么差。

那如果不通过标签来进行判断呢，输出是否会有不同？

或者说，并无指定标签的位置，而是将它们错落分布呢？

接下来看看不同的情况，会得到什么样的结果。

In [ ]:
def few_shot_sentiment_classification(input_text):
    few_shot_prompt = PromptTemplate(
        input_variables=["input_text"],
        template="""
        文本：欲穷千里目，更上一层楼。

        文本：风飒飒兮木萧萧，思公子兮徒离忧。

        文本：袅袅兮秋风，洞庭波兮木叶下。

        文本：{input_text}
        情感：
        """
    )

    chain = few_shot_prompt | llm
    result = chain.invoke(input_text).content

    return result

test_text = "夜中不能寐，起坐弹鸣琴。"
result = few_shot_sentiment_classification(test_text)
print(f"输入：{test_text}")
print(f"情感预测：{result}")

In [ ]:
def few_shot_sentiment_classification(input_text):
    few_shot_prompt = PromptTemplate(
        input_variables=["input_text"],
        template="""
        力拔山兮气盖世，时不利兮骓不逝。消极

        积极。邂逅相遇，与子偕臧。

        匪女之为美，美人之贻。
        积极

        文本：{input_text}
        情感：
        """
    )

    chain = few_shot_prompt | llm
    result = chain.invoke(input_text).content

    return result

test_text = "袅袅兮秋风，洞庭波兮木叶下。"
result = few_shot_sentiment_classification(test_text)
print(f"输入：{test_text}")
print(f"情感预测：{result}")

效果到底如何，我想你们在代码运行中可以得到答案了，那么这就引发了我们的思考：要如何编写 few-shot prompt，才能让模型更好的输出我们想要的结果？

许多研究表明，构建上下文示例的提示格式、训练示例和示例顺序的选择可以导致性能截然不同，从接近随机猜测到接近 SoTA，这也意味着，优秀的 few-shot prompt 可以极大提高模型性能，甚至可以接近 SoTA！

[这篇论文](https://arxiv.org/abs/2102.09690)调查了 few-shot prompt 的情况，并提出 LLM 的几种偏差导致了如此高的方差：

1. Majority Label Bias，如果示例中的标签分布不平衡，则存在多数标签偏差影响性能；
2. Recency Bias，模型可能对示例中的标签位置敏感，尤其是末尾标签；
3. Common Token Bias，模型倾向于生成常见 tokens 而不是罕见但正确的 tokens。

所以，我们在写 few-shot prompt 时，最好做到：指令明确、标签分布均匀、示例顺序随机化、表达多样化。

下面我们来看看如何编写较好的 few-shot prompt。

In [ ]:
def few_shot_sentiment_classification(input_text):
    few_shot_prompt = PromptTemplate(
        input_variables=["input_text"],
        template="""
        任务：请帮我对提供的文本进行情感分类，输出文本的情感为积极或消极。

        示例：
        文本：力拔山兮气盖世，时不利兮骓不逝。
        情感：消极

        文本：邂逅相遇，与子偕臧。
        情感：积极

        文本：匪女之为美，美人之贻。
        情感：积极

        文本：袅袅兮秋风，洞庭波兮木叶下。
        情感：消极

        请预测下面的文本的情感：
        文本：{input_text}
        情感：
        """
    )

    chain = few_shot_prompt | llm
    result = chain.invoke(input_text).content

    return result

test_text = "风飒飒兮木萧萧，思公子兮徒离忧。"
result = few_shot_sentiment_classification(test_text)
print(f"输入：{test_text}")
print(f"情感预测：{result}")

这个 prompt 的指令非常明确，并且，积极与消极的标签都分别选了两个，打乱了出现的顺序，至于表达的多样化，诗词的表达本身就非常多样化。

这里也有很多情况可以探索和优化，比如说标签数据实在拿不到平衡的分布要怎么做？校准输出概率使其均匀分布，减少模型对特定标签的偏好？这些有兴趣的读者都可以自行探索。